# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION =
    { 'class': 'SimpleStrategy', 'replication_factor' : 1}""")

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = ("""SELECT artist, song_title, songs_length
             FROM artist_songs
             WHERE sessionId = '338' AND itemInSession = '4'""")

try:
    session.execute("DROP TABLE IF EXISTS artist_songs")
except Exception as e:
    print(e)


query = ("CREATE TABLE IF NOT EXISTS artist_songs")
query = query + ("(sessionId varchar, itemInSession varchar, artist text, song_title text, songs_length varchar, PRIMARY KEY((sessionId, itemInSession), artist , song_title, songs_length))")
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO artist_songs (sessionId, itemInSession, artist, song_title, songs_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[8], line[3], line[0], line[9], line[5]))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song_title, row.songs_length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [18]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = ("""SELECT artist, song_title, firstname_user, lastname_user 
             FROM artist_song_username
             WHERE user_id = '10' AND session_id = '182'""")

                    

In [22]:
try:
    session.execute("DROP TABLE IF EXISTS artist_song_username")
except Exception as e:
    print(e)
    
try:
    query = ("CREATE TABLE IF NOT EXISTS artist_song_username")
    query = query + ("(session_id varchar, itemInSession varchar, \
                        user_id varchar, firstname_user text, lastname_user text,\
                        artist text, song_title text, \
                        PRIMARY KEY ((session_id, user_id), itemInSession))")
    session.execute(query)
except Exception as e:
    print(e)
    

In [23]:
file = ('event_datafile_new.csv')
with open (file, encoding= 'utf-8') as f:
    csvreader = csv.reader(f)
    next (csvreader)
    for line in csvreader:
        
        query = "INSERT INTO artist_song_username (session_id, itemInSession, user_id, firstname_user, lastname_user, artist, song_title)"
        query = query + ("VALUES (%s, %s, %s, %s, %s, %s, %s)")
        session.execute(query, (line[8], line[3], line[10], line[1], line[4], line[0], line[9]))


In [24]:
try:
    rows= session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print(row[0], row[1], row[2], row[3])

                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [31]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = ("""SELECT firstname_user, lastname_user 
             FROM username_song 
             WHERE song_title = 'All Hands Against His Own' ;""")

In [25]:
try:
    session.execute("DROP TABLE IF EXISTS username_song ;")
except Exception as e:
    print(e)
    
try:
    session.execute("""CREATE TABLE IF NOT EXISTS username_song (song_title text, session_id varchar,
                            itemInSession varchar, firstname_user text, lastname_user text,
                            PRIMARY KEY ((song_title), session_id, itemInSession))""")
except Exception as e:
    print(e)

In [29]:
with open(file, encoding= 'utf-8') as f:
    csvreader = csv.reader(f)
    next (csvreader)
    for line in csvreader:
        query = "INSERT INTO username_song (song_title, session_id, itemInSession, firstname_user, lastname_user)" 
        query = query + ("VALUES (%s, %s, %s, %s, %s)")
        session.execute(query , (line[9], line[8], line[3], line[1], line[4]))

In [32]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname_user, row.lastname_user)

Sara Johnson
Jacqueline Lynch
Tegan Levine


### Drop the tables before closing out the sessions

In [35]:
try:
    session.execute("DROP TABLE IF EXISTS artist_songs;")
    session.execute("DROP TABLE IF EXISTS artist_song_username;") 
    session.execute("DROP TABLE IF EXISTS username_song;")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()